DAEN 690- CAPSTONE PROJECT- TEAM PATRIOTS

TEAM MEMBERS

1) SHIVA RAM KAUSHIL PABBA
2) ANDREW BLOOM
3) DODDANAIK BASAVARAJ VAKKUND
4) JAHNAVI JONNALAGADDA
======================
THIS NOTEBOOK CONTAINS ANALYSIS FOR PROBLEM STATEMENT-3

1) Find the frequency of units dispatched after X minutes of becomming available. X need to be established in minutes. For example, X may be 20-30 minutes?

=============
1) Input dataset : 'Incidents_by_Unit.xlsx'
2) Output datasets: 'incidentbyunit_frequency.csv'

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import csv

In [2]:
#Load the dataset
data = pd.read_excel('./Incidents_by_Unit.xlsx')

In [3]:
#Print size, columns and data type for columns
print(data.head(5).to_string())
print(data.shape)
print(data.columns)
print(data.dtypes)

   Incident # Event Type Code        Event Type Event Category Unit Id Unit Type          Dispatched           Available
0  E180010058           GASIN  GAS LEAK: INSIDE              F    E425    Engine 2018-01-01 00:28:58 2018-01-01 00:39:37
1  E180010213             BLS    BLS EMERGENCY               E    E417    Engine 2018-01-01 01:33:05 2018-01-01 01:47:20
2  E180010457             ALS    ALS EMERGENCY               E    E405    Engine 2018-01-01 03:19:42 2018-01-01 03:30:43
3  E180010484             ALS    ALS EMERGENCY               E    M414     Medic 2018-01-01 03:43:05 2018-01-01 04:33:11
4  E180010490          ALRMFD        FIRE ALARM              F    E429    Engine 2018-01-01 03:46:47 2018-01-01 03:57:05
(636670, 8)
Index(['Incident #', 'Event Type Code', 'Event Type', 'Event Category',
       'Unit Id', 'Unit Type', 'Dispatched', 'Available'],
      dtype='object')
Incident #                 object
Event Type Code            object
Event Type                 object
Event C

In [4]:
#Dropping test and misc event Categories 
data.drop(data[data['Event Category'] == "T"].index, inplace = True) 

data.drop(data[data['Event Category'] == "N"].index, inplace = True) 
data = data.dropna()

In [5]:
data["Unit Id"].nunique()

322

In [6]:
df = data[["Incident #","Event Category","Unit Id","Dispatched","Available"]]

In [7]:
#sort by event category, unit id and dispatched date
df = df.sort_values(by=['Event Category', "Unit Id","Dispatched"])
df = df.reset_index()

In [8]:
df = df.drop(columns=["index"])

In [9]:
df

,Incident #,Event Category,Unit Id,Dispatched,Available
0,E180050580,E,A401,2018-01-05 07:58:13,2018-01-05 08:01:23
1,E180051059,E,A401,2018-01-05 11:26:27,2018-01-05 12:54:41
2,E180051520,E,A401,2018-01-05 14:35:44,2018-01-05 16:21:32
3,E180052714,E,A401,2018-01-05 23:02:16,2018-01-06 01:12:19
4,E180060182,E,A401,2018-01-06 02:17:25,2018-01-06 02:42:25
...,...,...,...,...,...
633713,E181460609,P,VC421,2018-05-26 06:20:55,2018-05-26 10:28:17
633714,E181522131,P,VC421,2018-06-01 15:31:20,2018-06-01 23:03:44
633715,E181540840,P,VC421,2018-06-03 10:11:41,2018-06-03 15:05:24
633716,E190410580,P,VC421,2019-02-10 08:52:07,2019-02-10 10:18:29


In [10]:
#Add new column to dataframe for difference
df['Dispatch_Available_diff'] = np.nan 
#  only take diffs where next row is of the same group
df['Dispatch_Available_diff'] = np.where(df["Unit Id"] == df["Unit Id"].shift(1), df["Dispatched"]-df["Available"].shift(1), 0)

In [11]:
# function to convert days to hours
df['Dispatch_Available_diff'] = pd.to_timedelta(df['Dispatch_Available_diff'])
def f(x):
    ts = x.total_seconds()
    hours, remainder = divmod(ts, 3600)
    minutes, seconds = divmod(remainder, 60)
    return ('{}:{:02d}:{:02d}').format(int(hours), int(minutes), int(seconds)) 

df['Dispatch_Available_diff_hours'] = df['Dispatch_Available_diff'].apply(f)


In [12]:
df

,Incident #,Event Category,Unit Id,Dispatched,Available,Dispatch_Available_diff,Dispatch_Available_diff_hours
0,E180050580,E,A401,2018-01-05 07:58:13,2018-01-05 08:01:23,0 days 00:00:00,0:00:00
1,E180051059,E,A401,2018-01-05 11:26:27,2018-01-05 12:54:41,0 days 03:25:04,3:25:04
2,E180051520,E,A401,2018-01-05 14:35:44,2018-01-05 16:21:32,0 days 01:41:03,1:41:03
3,E180052714,E,A401,2018-01-05 23:02:16,2018-01-06 01:12:19,0 days 06:40:44,6:40:44
4,E180060182,E,A401,2018-01-06 02:17:25,2018-01-06 02:42:25,0 days 01:05:06,1:05:06
...,...,...,...,...,...,...,...
633713,E181460609,P,VC421,2018-05-26 06:20:55,2018-05-26 10:28:17,0 days 00:00:00,0:00:00
633714,E181522131,P,VC421,2018-06-01 15:31:20,2018-06-01 23:03:44,6 days 05:03:03,149:03:03
633715,E181540840,P,VC421,2018-06-03 10:11:41,2018-06-03 15:05:24,1 days 11:07:57,35:07:57
633716,E190410580,P,VC421,2019-02-10 08:52:07,2019-02-10 10:18:29,251 days 17:46:43,6041:46:43


In [13]:
# converting HH.MM.SS to minutes and round of two decimals
df["Dispatch_Available_diff_min"]= df['Dispatch_Available_diff_hours'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]) + int(x[2])/60)
df["Dispatch_Available_diff_min"]= round(df.Dispatch_Available_diff_min,2)

In [15]:
df['dispatched_date'] = [d.date() for d in df['Dispatched']]

In [16]:
df.head(10)

,Incident #,Event Category,Unit Id,Dispatched,Available,Dispatch_Available_diff,Dispatch_Available_diff_hours,Dispatch_Available_diff_min,dispatched_date
0,E180050580,E,A401,2018-01-05 07:58:13,2018-01-05 08:01:23,0 days 00:00:00,0:00:00,0.00,2018-01-05
1,E180051059,E,A401,2018-01-05 11:26:27,2018-01-05 12:54:41,0 days 03:25:04,3:25:04,205.07,2018-01-05
2,E180051520,E,A401,2018-01-05 14:35:44,2018-01-05 16:21:32,0 days 01:41:03,1:41:03,101.05,2018-01-05
3,E180052714,E,A401,2018-01-05 23:02:16,2018-01-06 01:12:19,0 days 06:40:44,6:40:44,400.73,2018-01-05
4,E180060182,E,A401,2018-01-06 02:17:25,2018-01-06 02:42:25,0 days 01:05:06,1:05:06,65.10,2018-01-06
5,E180160920,E,A401,2018-01-16 09:00:38,2018-01-16 10:12:30,10 days 06:18:13,246:18:13,14778.22,2018-01-16
6,E180161254,E,A401,2018-01-16 11:04:03,2018-01-16 12:48:32,0 days 00:51:33,0:51:33,51.55,2018-01-16
7,E180161861,E,A401,2018-01-16 14:32:32,2018-01-16 15:41:17,0 days 01:44:00,1:44:00,104.00,2018-01-16
8,E180262642,E,A401,2018-01-26 20:59:01,2018-01-26 21:39:44,10 days 05:17:44,245:17:44,14717.73,2018-01-26
9,E180262975,E,A401,2018-01-26 23:28:02,2018-01-27 00:36:33,0 days 01:48:18,1:48:18,108.30,2018-01-26


In [17]:
# dropping first row, since we dont know the diff between available and first dispatched
df.drop(df[df['Dispatch_Available_diff_min'] == 0].index, inplace = True) 

In [18]:
df

,Incident #,Event Category,Unit Id,Dispatched,Available,Dispatch_Available_diff,Dispatch_Available_diff_hours,Dispatch_Available_diff_min,dispatched_date
1,E180051059,E,A401,2018-01-05 11:26:27,2018-01-05 12:54:41,0 days 03:25:04,3:25:04,205.07,2018-01-05
2,E180051520,E,A401,2018-01-05 14:35:44,2018-01-05 16:21:32,0 days 01:41:03,1:41:03,101.05,2018-01-05
3,E180052714,E,A401,2018-01-05 23:02:16,2018-01-06 01:12:19,0 days 06:40:44,6:40:44,400.73,2018-01-05
4,E180060182,E,A401,2018-01-06 02:17:25,2018-01-06 02:42:25,0 days 01:05:06,1:05:06,65.10,2018-01-06
5,E180160920,E,A401,2018-01-16 09:00:38,2018-01-16 10:12:30,10 days 06:18:13,246:18:13,14778.22,2018-01-16
...,...,...,...,...,...,...,...,...,...
633711,E192500462,P,VC414,2019-09-07 06:15:45,2019-09-07 13:39:27,91 days 05:20:22,2189:20:22,131360.37,2019-09-07
633714,E181522131,P,VC421,2018-06-01 15:31:20,2018-06-01 23:03:44,6 days 05:03:03,149:03:03,8943.05,2018-06-01
633715,E181540840,P,VC421,2018-06-03 10:11:41,2018-06-03 15:05:24,1 days 11:07:57,35:07:57,2107.95,2018-06-03
633716,E190410580,P,VC421,2019-02-10 08:52:07,2019-02-10 10:18:29,251 days 17:46:43,6041:46:43,362506.72,2019-02-10


In [19]:
# considering only minum frequency in each day
grouped_df = df.groupby(["Event Category","Unit Id","dispatched_date"]).min()

In [20]:
grouped_df

Incident #          Dispatched  \
Event Category Unit Id dispatched_date                                   
E              A401    2018-01-05       E180051059 2018-01-05 11:26:27   
                       2018-01-06       E180060182 2018-01-06 02:17:25   
                       2018-01-16       E180160920 2018-01-16 09:00:38   
                       2018-01-26       E180262642 2018-01-26 20:59:01   
                       2018-01-29       E180291284 2018-01-29 11:54:09   
...                                            ...                 ...   
P              VC414   2019-09-07       E192500462 2019-09-07 06:15:45   
               VC421   2018-06-01       E181522131 2018-06-01 15:31:20   
                       2018-06-03       E181540840 2018-06-03 10:11:41   
                       2019-02-10       E190410580 2019-02-10 08:52:07   
                       2019-02-20       E190511243 2019-02-20 15:03:24   

                                                 Available  \
Event Category Unit Id dispatched_date                       
E              A401    2018-01-05      2018-01-05 12:54:41   
                       2018-01-06      2018-01-06 02:42:25   
                       2018-01-16      2018-01-16 10:12:30   
                       2018-01-26      2018-01-26 21:39:44   
                       2018-01-29      2018-01-29 11:58:13   
...                                                    ...   
P              VC414   2019-09-07      2019-09-07 13:39:27   
               VC421   2018-06-01      2018-06-01 23:03:44   
                       2018-06-03      2018-06-03 15:05:24   
                       2019-02-10      2019-02-10 10:18:29   
                       2019-02-20      2019-02-20 15:22:01   

                                       Dispatch_Available_diff  \
Event Category Unit Id dispatched_date                           
E              A401    2018-01-05              0 days 01:41:03   
                       2018-01-06              0 days 01:05:06   
                       2018-01-16              0 days 00:51:33   
                       2018-01-26              0 days 01:48:18   
                       2018-01-29              0 days 02:21:18   
...                                                        ...   
P              VC414   2019-09-07             91 days 05:20:22   
               VC421   2018-06-01              6 days 05:03:03   
                       2018-06-03              1 days 11:07:57   
                       2019-02-10            251 days 17:46:43   
                       2019-02-20             10 days 04:44:55   

                                       Dispatch_Available_diff_hours  \
Event Category Unit Id dispatched_date                                 
E              A401    2018-01-05                            1:41:03   
                       2018-01-06                            1:05:06   
                       2018-01-16                            0:51:33   
                       2018-01-26                            1:48:18   
                       2018-01-29                            2:21:18   
...                                                              ...   
P              VC414   2019-09-07                         2189:20:22   
               VC421   2018-06-01                          149:03:03   
                       2018-06-03                           35:07:57   
                       2019-02-10                         6041:46:43   
                       2019-02-20                          244:44:55   

                                        Dispatch_Available_diff_min  
Event Category Unit Id dispatched_date                               
E              A401    2018-01-05                            101.05  
                       2018-01-06                             65.10  
                       2018-01-16                             51.55  
                       2018-01-26                            108.30  
                       2018-01-29                     

In [21]:
# Calculating min, max, average frequencies for each unit and event category
all_together = (grouped_df.groupby(["Unit Id","Event Category"])
                  .agg({'Dispatch_Available_diff_min': [np.mean, np.min, np.max]})
                  .rename(columns={'mean': 'avg_duration_mins', 'amin': 'min_duration_mins', 'amax': 'max_duration_mins'}))

In [22]:
all_together

Dispatch_Available_diff_min                    \
                                 avg_duration_mins min_duration_mins   
Unit Id Event Category                                                 
A401    E                              4120.839733              0.23   
        F                            224936.500000          78524.70   
        P                             35914.683548            102.98   
A402    E                              1797.270481              0.03   
        F                            100405.405000             12.38   
...                                            ...               ...   
VC417   E                             31674.692667             56.07   
        F                             50241.387778            102.45   
VC421   E                             31217.307143              0.98   
        F                            115502.153333           9787.85   
        P                             97060.660000           2107.95   

                                          
                       max_duration_mins  
Unit Id Event Category                    
A401    E                       45217.07  
        F                      371353.58  
        P                      234433.85  
A402    E                       23711.20  
        F                      273717.65  
...                                  ...  
VC417   E                       96751.97  
        F                      103091.43  
VC421   E                      209065.82  
        F                      210165.57  
        P                      362506.72  

[739 rows x 3 columns]

In [23]:
all_together.reset_index().head()

Unit Id Event Category Dispatch_Available_diff_min                    \
                                   avg_duration_mins min_duration_mins   
0    A401              E                 4120.839733              0.23   
1    A401              F               224936.500000          78524.70   
2    A401              P                35914.683548            102.98   
3    A402              E                 1797.270481              0.03   
4    A402              F               100405.405000             12.38   

                     
  max_duration_mins  
0          45217.07  
1         371353.58  
2         234433.85  
3          23711.20  
4         273717.65

In [25]:
grouped_df.reset_index().to_csv("./incidentbyunit_frequency.csv")